In [1]:
import Parsing as ps
import numpy as np
import re
import os
import tetra_dude as td
import matplotlib.pyplot as plt

from numpy import *
from operator import itemgetter, attrgetter, methodcaller
from numpy import *
%matplotlib inline 

import keras

from keras.preprocessing import sequence
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, RMSprop, Adagrad, Adam, Adadelta
from keras.utils import np_utils

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [6]:
clean_dict = ps.clean('./',92)
new = ps.noisy_head_info('./',1211177, clean_dict)
PI = ps.make_PI(new)

with open("Illumina_LinErr_100_fold1.fasta", "w") as f:
    for i in range(len(new)):
        data = "%s\n"%new[i][4]
        f.write(data)
    
nb_classes = 4
nt_order = "ATGCN\n"
#PI = array([[0.9938, 0.0005, 0.0049, 0.0008],
#          [0.0030, 0.9880, 0.0034, 0.0056],
#          [0.0084, 0.0053, 0.9816, 0.0046],
#          [0.0006, 0.0042, 0.0008, 0.9944]])
H = linalg.inv(PI)
LAMBDA = array([[0, 1, 1, 1],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [1, 1, 1, 0]])
S = zeros((nb_classes, nb_classes**nb_classes),dtype=int)
for s in range(1,nb_classes**nb_classes):
    for x in range(4):
        S[x][s] = S[x][s-1]
    
    if S[0][s] != 3:
        S[0][s] += 1
        continue
    else:
        S[0][s] = 0
        
    if S[1][s] != 3:
        S[1][s] += 1
        continue
    else:
        S[1][s] = 0
        
    if S[2][s] != 3:
        S[2][s] += 1
        continue
    else:
        S[2][s] = 0
        
    if S[3][s] != 3:
        S[3][s] += 1
        continue
    else:
        print "ERROR: S index out of bound!\n"
RHO   = zeros((nb_classes, nb_classes**nb_classes),dtype=float)
RHO_R = zeros((nb_classes, nb_classes*nb_classes),dtype=float)
for s in range(nb_classes**nb_classes):
    for x in range(4):
        RHO[x][s] = (PI[x][0]*LAMBDA[x][S[0][s]] + PI[x][1]*LAMBDA[x][S[1][s]] +
                     PI[x][2]*LAMBDA[x][S[2][s]] + PI[x][3]*LAMBDA[x][S[3][s]])
for s in range(nb_classes*nb_classes):
    for x in range(4):
        RHO_R[x][s] = PI[x][int(s/4)]*LAMBDA[x][s%4]
L=dot(H,RHO) 
L_new=-L+amax(L)     # A new loss matrix
L_R = dot(H,RHO_R)
L_R_new=-L_R+amax(L_R)
k_max=26

Pre-processing starts...
Pre-processing is 0.0% done..
Pre-processing is 16.5% done..
Pre-processing is 33.0% done..
Pre-processing is 49.5% done..
Pre-processing is 66.0% done..
Pre-processing is 82.5% done..
Pre-processing is 99.0% done..
Reshaping...
Pre-processing is completed!!
Making PI starts...
Making PI is 0.0% done..
Making PI is 16.8% done..
Making PI is 33.6% done..
Making PI is 50.4% done..
Making PI is 67.2% done..
Making PI is 84.0% done..
Making PI is completed!!


In [ ]:
with open("./fold1.fasta",'r') as tmp:
    tmp_ = tmp.readlines()[:50]
    for i in range(len(tmp_)):
        print tmp_[i]

In [7]:
LOG = open("log.txt","w")
fold_name = ['fold1']#, 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9']
file_stub = 'Illumina_LinErr_'
file_name= []
for i in range(len(fold_name)):
    for j in range(2,10):
            continue
            name = '%s0%s0_%s' % (file_stub, str(j), fold_name[i])
            file_name.append(name)
    name = '%s100_%s' % (file_stub, fold_name[i])
    file_name.append(name)

for k in range(5,6):
    for name in file_name:
        name_out = name + '_ND' + str(k)
        td.PRINT(LOG, "%s\tPROCESSING START" % name_out)
        f_in = open("%s.fasta" % name, "r")
        lines = f_in.readlines()
        f_in.close()

    # -----------------------------------------------------
    #   For directly generating data for Neural DUDE
    # -----------------------------------------------------
        lines,C,Y,Y_R = td.make_data_for_ndude(lines,k,L_new,L_R_new,nt_order)
    # -----------------------------------------------------
    # -----------------------------------------------------
    #   Defining neural networks
    # -----------------------------------------------------
        model1=Sequential()
        model1.add(Dense(2000,input_dim=2*k*(nb_classes),init='he_normal'))
        model1.add(Activation('relu'))
        model1.add(Dense(2000,init='he_normal'))
        model1.add(Activation('relu'))
        model1.add(Dense(2000,init='he_normal'))
        model1.add(Activation('relu'))
        model1.add(Dense(256,init='he_normal'))
        model1.add(Activation('softmax'))

        rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-06,clipnorm=1.5)
        adagrad=Adagrad(clipnorm=1.5)
        adam=Adam()
        adadelta=Adadelta()
        sgd=SGD(lr=0.01,decay=1e-6,momentum=0.95, nesterov=True, clipnorm=1.0)

        model1.compile(loss='poisson', optimizer=adam)

        td.PRINT(LOG,'Neural DUDE fitting...')
        model1.fit(C,Y,nb_epoch=50,batch_size=500,show_accuracy=False, verbose=1)
        td.PRINT(LOG,'Neural DUDE fitting DONE')

    # -----------------------------------------------------
    # -----------------------------------------------------
    # -----------------------------------------------------
        model4=Sequential()
        model4.add(Dense(2000,input_dim=2*k*(nb_classes),init='he_normal'))
        model4.add(Activation('relu'))
        model4.add(Dense(2000,init='he_normal'))
        model4.add(Activation('relu'))
        model4.add(Dense(2000,init='he_normal'))
        model4.add(Activation('relu'))
        model4.add(Dense(256,init='he_normal'))
        model4.add(Activation('softmax'))

        model4.compile(loss='poisson', optimizer=adam)

        td.PRINT(LOG,'Neural DUDE2 fitting...')
        model4.fit(C,Y,nb_epoch=50,batch_size=500,show_accuracy=False, verbose=1)
        td.PRINT(LOG,'Neural DUDE2 fitting DONE')

    # -----------------------------------------------------
        model2=Graph()
        model2.add_input(name='input', input_shape=(2*k*(nb_classes),))
        model2.add_node(Dense(400,init='he_normal'), name='layer1', input='input')
        model2.add_node(Activation('relu'), name='activation1', input='layer1')
        model2.add_node(Dense(400,init='he_normal'), name='layer2', input='activation1')
        model2.add_node(Activation('relu'), name='activation2', input='layer2')
        model2.add_node(Dense(400,init='he_normal'), name='layer3', input='activation2')
        model2.add_node(Activation('relu'), name='activation3', input='layer3')
        model2.add_node(Dense(4,init='he_normal'), name='layer4_0', input='activation3')
        model2.add_node(Dense(4,init='he_normal'), name='layer4_1', input='activation3')
        model2.add_node(Dense(4,init='he_normal'), name='layer4_2', input='activation3')
        model2.add_node(Dense(4,init='he_normal'), name='layer4_3', input='activation3')
        model2.add_node(Activation('softmax'), name='activation4_0', input='layer4_0')
        model2.add_node(Activation('softmax'), name='activation4_1', input='layer4_1')
        model2.add_node(Activation('softmax'), name='activation4_2', input='layer4_2')
        model2.add_node(Activation('softmax'), name='activation4_3', input='layer4_3')
        model2.add_output(name='output0', input='activation4_0')
        model2.add_output(name='output1', input='activation4_1')
        model2.add_output(name='output2', input='activation4_2')
        model2.add_output(name='output3', input='activation4_3')

        rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-06,clipnorm=1.5)
        adagrad=Adagrad(clipnorm=1.5)
        adam=Adam()
        adadelta=Adadelta()
        sgd=SGD(lr=0.01,decay=1e-6,momentum=0.95, nesterov=True, clipnorm=1.0)

        model2.compile(loss={'output0':'poisson','output1':'poisson','output2':'poisson','output3':'poisson'}, 
                       optimizer=adam)

        #td.PRINT(LOG,'Neural DUDE reduced fitting...')
        #model2.fit({'input':C,'output0':Y_R[:,0:4],'output1':Y_R[:,4:8],'output2':Y_R[:,8:12],'output3':Y_R[:,12:16]}, 
        #           nb_epoch=20,batch_size=500,verbose=1)
        #td.PRINT(LOG,'Neural DUDE reduced fitting DONE')

    # -----------------------------------------------------
    # -----------------------------------------------------
        model3=Graph()
        model3.add_input(name='input', input_shape=(2*k*(nb_classes),))
        model3.add_node(Dense(400,init='he_normal'), name='layer1', input='input')
        model3.add_node(Activation('relu'), name='activation1', input='layer1')
        model3.add_node(Dense(400,init='he_normal'), name='layer2', input='activation1')
        model3.add_node(Activation('relu'), name='activation2', input='layer2')
        model3.add_node(Dense(400,init='he_normal'), name='layer3', input='activation2')
        model3.add_node(Activation('relu'), name='activation3', input='layer3')
        model3.add_node(Dense(4,init='he_normal'), name='layer4_0', input='activation3')
        model3.add_node(Dense(4,init='he_normal'), name='layer4_1', input='activation3')
        model3.add_node(Dense(4,init='he_normal'), name='layer4_2', input='activation3')
        model3.add_node(Dense(4,init='he_normal'), name='layer4_3', input='activation3')
        model3.add_node(Activation('softmax'), name='activation4_0', input='layer4_0')
        model3.add_node(Activation('softmax'), name='activation4_1', input='layer4_1')
        model3.add_node(Activation('softmax'), name='activation4_2', input='layer4_2')
        model3.add_node(Activation('softmax'), name='activation4_3', input='layer4_3')
        model3.add_output(name='output0', input='activation4_0')
        model3.add_output(name='output1', input='activation4_1')
        model3.add_output(name='output2', input='activation4_2')
        model3.add_output(name='output3', input='activation4_3')

        model3.compile(loss={'output0':'poisson','output1':'poisson','output2':'poisson','output3':'poisson'}, 
                       optimizer=adam)

        #td.PRINT(LOG,'Neural DUDE reduced2 fitting...')
        #model3.fit({'input':C,'output0':Y_R[:,0:4],'output1':Y_R[:,4:8],'output2':Y_R[:,8:12],'output3':Y_R[:,12:16]}, 
        #           nb_epoch=20,batch_size=500,verbose=1)
        #td.PRINT(LOG,'Neural DUDE reduced2 fitting DONE')

    # -----------------------------------------------------
        

        td.PRINT(LOG,'Neural DUDE denoising...')
        f_out1 = open("%s_1.fasta" % name_out, "w")
        pred_class1=model1.predict_classes(C, batch_size=200, verbose=0)
        td.denoise_with_s(f_out1,lines,pred_class1,k,nt_order)
        f_out1.close()
        td.PRINT(LOG,'Neural DUDE denoisig DONE')
        
        td.PRINT(LOG,'Neural DUDE2 denoising...')
        f_out4 = open("%s_2.fasta" % name_out, "w")
        pred_class4=model4.predict_classes(C, batch_size=200, verbose=0)
        td.denoise_with_s(f_out4,lines,pred_class4,k,nt_order)
        f_out4.close()
        td.PRINT(LOG,'Neural DUDE denoisig DONE')
        
        '''
        td.PRINT(LOG,'Neural DUDE reduced denoising...')
        f_out2 = open("%s_r.fasta" % name_out, "w")
        pred2=model2.predict({'input':C}, batch_size=200, verbose=0)
        pred_class2 = td.s_R_preprocess(pred2, k)
        td.denoise_with_s_R(f_out2,lines,pred_class2,k,nt_order)
        f_out2.close()
        td.PRINT(LOG,'Neural DUDE reduced denoisig DONE')
        
        td.PRINT(LOG,'Neural DUDE reduced2 denoising...')
        f_out3 = open("%s_r2.fasta" % name_out, "w")
        pred3=model3.predict({'input':C}, batch_size=200, verbose=0)
        pred_class3 = td.s_R_preprocess(pred3, k)
        td.denoise_with_s_R(f_out3,lines,pred_class3,k,nt_order)
        f_out3.close()
        td.PRINT(LOG,'Neural DUDE reduced2 denoisig DONE')

        td.PRINT(LOG,'DUDE denoising...')
        f_out4 = open("%s_DS5_msw.fasta" % name_out, "w")
        s_hat= td.dude(lines,k,L) 
        td.denoise_with_s(f_out4,lines,s_hat,k,nt_order)
        f_out4.close()
        td.PRINT(LOG,'DUDE denoisig DONE\n')
        '''


2016-11-17 15:17:39.083984	Illumina_LinErr_100_fold1_ND5	PROCESSING START


MemoryError: 